In [14]:
import torch
from models.dpirUnet import NNclass2
import math

In [15]:
x=torch.load('/export/project/zihao/potential_exp/gspnp_joint/2.55/epoch_9.pt')

In [16]:
x.keys()

odict_keys(['sigmaFactor', 'f.tau', 'f.lamb', 'f.rObj.network.m_head.weight', 'f.rObj.network.m_down1.0.res.0.weight', 'f.rObj.network.m_down1.0.res.2.weight', 'f.rObj.network.m_down1.1.res.0.weight', 'f.rObj.network.m_down1.1.res.2.weight', 'f.rObj.network.m_down1.2.weight', 'f.rObj.network.m_down2.0.res.0.weight', 'f.rObj.network.m_down2.0.res.2.weight', 'f.rObj.network.m_down2.1.res.0.weight', 'f.rObj.network.m_down2.1.res.2.weight', 'f.rObj.network.m_down2.2.weight', 'f.rObj.network.m_down3.0.res.0.weight', 'f.rObj.network.m_down3.0.res.2.weight', 'f.rObj.network.m_down3.1.res.0.weight', 'f.rObj.network.m_down3.1.res.2.weight', 'f.rObj.network.m_down3.2.weight', 'f.rObj.network.m_body.0.res.0.weight', 'f.rObj.network.m_body.0.res.2.weight', 'f.rObj.network.m_body.1.res.0.weight', 'f.rObj.network.m_body.1.res.2.weight', 'f.rObj.network.m_up3.0.weight', 'f.rObj.network.m_up3.1.res.0.weight', 'f.rObj.network.m_up3.1.res.2.weight', 'f.rObj.network.m_up3.2.res.0.weight', 'f.rObj.network

In [20]:
new_state_dict = {}
for k, v in x.items():
    if 'f.rObj.network.' in k:
        name = k.replace('f.rObj.network.', '')
        new_state_dict[name] = v

In [21]:
torch.save(new_state_dict, 'pretrained/k-all_sigma-2.55.pt')

In [22]:
new_state_dict.keys()

dict_keys(['m_head.weight', 'm_down1.0.res.0.weight', 'm_down1.0.res.2.weight', 'm_down1.1.res.0.weight', 'm_down1.1.res.2.weight', 'm_down1.2.weight', 'm_down2.0.res.0.weight', 'm_down2.0.res.2.weight', 'm_down2.1.res.0.weight', 'm_down2.1.res.2.weight', 'm_down2.2.weight', 'm_down3.0.res.0.weight', 'm_down3.0.res.2.weight', 'm_down3.1.res.0.weight', 'm_down3.1.res.2.weight', 'm_down3.2.weight', 'm_body.0.res.0.weight', 'm_body.0.res.2.weight', 'm_body.1.res.0.weight', 'm_body.1.res.2.weight', 'm_up3.0.weight', 'm_up3.1.res.0.weight', 'm_up3.1.res.2.weight', 'm_up3.2.res.0.weight', 'm_up3.2.res.2.weight', 'm_up2.0.weight', 'm_up2.1.res.0.weight', 'm_up2.1.res.2.weight', 'm_up2.2.res.0.weight', 'm_up2.2.res.2.weight', 'm_up1.0.weight', 'm_up1.1.res.0.weight', 'm_up1.1.res.2.weight', 'm_up1.2.res.0.weight', 'm_up1.2.res.2.weight', 'm_tail.weight'])